In [ ]:
import openpyxl
import os
import numpy as np
import scipy  
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn import metrics
import random
import gc 

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential

In [ ]:
chosen="B5"  

In [ ]:
if chosen=="B0":
    imagesize=224
elif chosen=="B1":
    imagesize=240
elif chosen=="B2":
    imagesie=260
elif chosen=="B3":
    imagesize=300
elif chosen=="B4":
    imagesize=380
elif chosen=="B5":
    imagesize=456
elif chosen=="B6":
    imagesize=528
elif chosen=="B7":
    imagesize=600

In [ ]:
date=20230417

In [ ]:
paths=['./result/%d/FFT'%(date),
       './result/%d/FFT/bestmodel'%(date),
       './result/%d/FFT/finalmodel'%(date),
       './result/%d/bestmodel'%(date),
       './result/%d/finalmodel'%(date)       
      ]

In [ ]:
for path in paths:
    os.makedirs(path)

In [ ]:
#loading lists of png files

In [ ]:
train_ds = tf.data.Dataset.list_files('/home/hk/Research/CXRage/dataset0104/train/*.png', shuffle=False)  

In [ ]:
image_count1 = len(train_ds)
train_ds = train_ds.shuffle(image_count1, reshuffle_each_iteration=False)  

In [ ]:
val_ds = tf.data.Dataset.list_files('/home/hk/Research/CXRage/dataset0104/val/*.png', shuffle=False)  

In [ ]:
image_count2 = len(val_ds)
val_ds = val_ds.shuffle(image_count2, reshuffle_each_iteration=False)  

In [ ]:
test_ds = tf.data.Dataset.list_files('/home/hk/Research/CXRage/dataset0104/test/*.png', shuffle=False)  

In [ ]:
image_count3 = len(test_ds)
test_ds = test_ds.shuffle(image_count3, reshuffle_each_iteration=False)  

In [ ]:
def load_image(filename, augment=False): 
    raw = tf.io.read_file(filename)
    image = tf.image.decode_png(raw, channels=3)
    
    image.set_shape([512, 512, 3])  
    image = tf.image.resize(image, [imagesize, imagesize]) 
    
    if augment:
        image = tf.image.random_brightness(image, max_delta=25)
        image = tf.image.random_contrast(image, 0.7, 1.3)
        image = tf.clip_by_value(image, 0, 255)        
    return image 

In [ ]:
def get_label(filename):
    label = tf.strings.split(filename, sep=".")[-2]
    label = tf.strings.to_number(label, tf.float32)    
    label = tf.math.subtract(label, 11)
    label = tf.math.divide(label, 83)
    label = tf.math.add(label, 0.0001)
    return label

In [ ]:
def process_path_train(file_path):
    label = get_label(file_path)
    img = load_image(file_path, augment=True)
    return img, label

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    img = load_image(file_path)
    return img, label

In [ ]:
train_ds = train_ds.map(process_path_train, num_parallel_calls=tf.data.AUTOTUNE) 
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
img_augmentation = Sequential(
    [tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(-0.1, 0.1), fill_mode="constant"),   
     tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
     tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=(-0.05, 0.05), width_factor=(-0.05, 0.05), fill_mode="constant"),
     tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.05, 0.05), fill_mode="constant")
    ],
    name="img_augmentation")

In [ ]:
batch_size = 40

def configure_for_performance(ds):
    if chosen=="B0":
        ds = ds.cache() 
    ds = ds.shuffle(buffer_size=1000) 
    ds = ds.batch(batch_size, drop_remainder=True)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

In [ ]:
from tensorflow.keras.applications import EfficientNetB5

In [ ]:
cos_decay_ann = tf.keras.experimental.CosineDecayRestarts(initial_learning_rate=0.01, first_decay_steps=30, t_mul=2, m_mul=0.95, alpha=0.01)

In [ ]:
os.chdir("/home/hk/Research/CXRage/result/%d/"%(date))

In [ ]:
def build_model():
    inputs = layers.Input(shape=(imagesize, imagesize, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB5(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    
    droprate=0.3

    x = layers.Dropout(droprate, name="dropout1")(x)
    x = layers.Dense(256, kernel_initializer='he_normal', activation="relu")(x)   
    
    x = layers.Dropout(droprate, name="dropout2")(x)
    x = layers.Dense(128, kernel_initializer='he_normal', activation="relu")(x)  
    
    x = layers.Dropout(droprate, name="dropout3")(x)
    x = layers.Dense(64, kernel_initializer='he_normal', activation="relu")(x)  
    
    x = layers.Dropout(droprate, name="dropout4")(x)
    outputs = layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros', activation="linear", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=cos_decay_ann),
                  loss="mse", metrics=['mae', 'mse'])
    return model

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, min_delta= 0.0001) 
csv_logger = keras.callbacks.CSVLogger('./Model log.csv', append=False, separator=';')
checkpointer = keras.callbacks.ModelCheckpoint(filepath='/home/hk/Research/CXRage/result/%d/bestmodel'%(date), 
                                               verbose=1, 
                                               save_best_only=True, monitor='val_mae', mode='min')

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [ ]:
epochs = 1000
with mirrored_strategy.scope():
    model = build_model()     
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, 
                 callbacks=[early_stopping, csv_logger, checkpointer],
                 verbose=1) 

In [ ]:
model.save('/home/hk/Research/CXRage/result/%d/finalmodel'%(date))
model.save('CXRage%d.h5'%(date))

In [ ]:
hist1 = pd.DataFrame(hist.history)
hist1['epoch'] = hist.epoch
hist1.to_csv('history.csv', index=False, header=True)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
################################# Full fine-tuning #################################

In [ ]:
def unfreeze_model(model):
    for layer in model.layers:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

In [ ]:
cos_decay_ann = tf.keras.experimental.CosineDecayRestarts(initial_learning_rate=0.001, first_decay_steps=40, t_mul=2, m_mul=0.95, alpha=0.01)

In [ ]:
os.chdir("/home/hk/Research/CXRage/result/%d/FFT"%(date))

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, min_delta= 0.0001) 
csv_logger = keras.callbacks.CSVLogger('./Model log.csv', append=False, separator=';')
checkpointer = keras.callbacks.ModelCheckpoint(filepath='/home/hk/Research/CXRage/result/%d/FFT/bestmodel'%(date), 
                                               verbose=1, 
                                               save_best_only=True, monitor='val_mae', mode='min')

In [ ]:
epochs = 1000
with mirrored_strategy.scope(): 
    unfreeze_model(model)    
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=cos_decay_ann),
                   loss="mse", metrics=['mae', 'mse']) 

In [ ]:
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, 
                  callbacks=[early_stopping, csv_logger, checkpointer],
                  verbose=1) 

In [ ]:
model.save('/home/hk/Research/CXRage/result/%d/FFT/finalmodel'%(date))
model.save('CXRage%dFFT.h5'%(date))

In [ ]:
hist1 = pd.DataFrame(hist.history)
hist1['epoch'] = hist.epoch
hist1.to_csv('history.csv', index=False, header=True)